# Загрузка Pandas и очистка данных

In [174]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split



In [198]:

def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # переведём в нижний регистр названия столбцов
    df_output.columns = df_output.columns.str.lower()
    # заменим пробелы в названиях столбцов на символ нижний подчерк
    df_output.columns = df_output.columns.str.replace(' ', '_')
    
    
    # ################### 2. NAN ############################################################## 
    # создадим отделые колонки для строк с NAN
    df_output['number_of_reviews_is_nan'] = pd.isna(df_output['number_of_reviews']).astype('uint8')
    df_output['number_of_reviews'].fillna(0, inplace=True)
    
    df_output['cuisine_style_is_nan'] = pd.isna(df_output['cuisine_style']).astype('uint8')

    df_output['price_range_is_nan'] = df_output['price_range'].isna().astype('uint8')
   
    
    
    # ################### 3. Encoding ############################################################## 
    
    def clean_name(str_val):
        """
        Преобразует строку с названиями кухонь в список [list] названий кухонь.
        На входе:
            - строковая переменная, содержащая названия кухонь.
        На выходе:
            - список [list] названий кухонь.
        """
        if pd.isna(str_val): return ["Unknown"]
        str_val = str_val.strip('[]') # Отбрасываем скобки.
        str_val = str_val.replace("\'",'') # Убираем надоедливые кавычки '.
        str_val = str_val.split(", ") # Разбиваем строку по названиям кухонь.
        return str_val
    
    # Переведём данные из колонки cuisine_style к списку, пропуски заполним "Unknown"
    df_output["cuisine_style_list"] = df_output["cuisine_style"].apply(clean_name)
    

    # dummy variable prise-range исходя из распределения "$", "$$ - $$$", "$$$"
    price_range = {'$': 0, '$$ - $$$': 1, '$$$$': 2}
    df_output['price_range'] = df_output['price_range'].map(price_range)
    df_output['price_range'].fillna(df_output['price_range'].mean(), inplace=True) 
    
    # ################### 4. Feature Engineering ####################################################
    # количество кухонь в ресторане
    df_output['cuisine_style_count'] = df_output['cuisine_style_list'].apply(lambda x: 1 if len(x) == 0 else len(x))
    
    def reviews_date(str_rev):
        ''' Извлечение дат из колонки reviеws'''
        reviews_date = []
        for i in str_rev:
            if i == '[[], []]' or i == 0:
                reviews_date.append('')
            else:
                i = str(i).replace(']]', '')
                i = i.replace('"', "'")  # строка 177
                i = i.replace('nan', "'nan'")  # строка 5315
                i = i.replace("'", '')
                i = i.split('], [')[1]
                i = i.split(', ')
                reviews_date.append(i)
        return reviews_date
    
    df_output['reviews'].fillna(0, inplace=True)
    df_output['reviews_date'] = reviews_date(df_output.reviews)          
    df_output['reviews_date_1'] = df_output['reviews_date'].apply(lambda x: x[1] if len(x) == 2 else None)
    df_output['reviews_date_2'] = df_output['reviews_date'].apply(lambda x: x[0] if len(x) > 0 else None)
    
    # reviews_date_1 - дата первого отзыва
    # reviews_date_2 - дата второго отзыва
    df_output['reviews_date_1'] = pd.to_datetime(df_output['reviews_date_1'])
    df_output['reviews_date_2'] = pd.to_datetime(df_output['reviews_date_2'])

    # review_days_range - промежуток между отзывами в днях
    df_output['review_days_range'] = (df_output['reviews_date_2'] - df_output['reviews_date_2']).dt.days
    
    # days_untill_today - промежуток с последнего отзыва в днях
    df_output['days_untill_today'] = (datetime.now() - df_output['reviews_date_2']).dt.days
    
    # rest_city - количество ресторанов в городе
    df_output['rest_city'] = df_output['city'].map(df_output.groupby(['city'])['restaurant_id'].count().to_dict())
    
    # relative_rank - отношение ранга ресторана к количеству ресторанов в городе
    df_output['relative_rank'] = df_output['ranking'] / df_output['rest_city']
    
    # rew_city - количество отзывов в городе
    df_output['rew_city'] = df_output['city'].map(df_output.groupby(['city'])['number_of_reviews'].count().to_dict())
    
    # relative_rank_reviews - отношение ранга ресторана к количеству отзывов в городе
    df_output['relative_rank_reviews'] = df_output['ranking'] / df_output['rew_city']
    
    # какой-то ID(O_o???)
    df_output['id_ta'] = df_output['id_ta'].apply(lambda x: float(x[1:]))
 
    population = {'London': 8.982,'Paris': 2.148, 'Madrid': 6.642, 'Berlin': 3.769,  'Rome':2.873 , 'Prague': 1.309,
              'Lisbon': 0.504, 'Vienna': 1.897, 'Amsterdam': 0.821, 'Brussels': 0.174, 'Stockholm': 0.975, 
              'Budapest': 1.752, 'Warsaw': 1.708, 'Dublin': 1.388, 'Copenhagen': 0.602, 'Athens': 0.664, 
              'Edinburgh': 0.482, 'Oslo': 0.681, 'Helsinki': 0.631, 'Bratislava': 0.424, 'Luxembourg': 0.116, 
              'Ljubljana': 0.279, 'Munich': 1.472, 'Oporto': 0.214, 'Milan': 1.352, 'Barcelona': 5.575,
              'Zurich': 0.402, 'Lyon': 0.513, 'Hamburg': 1.899, 'Geneva': 0.499, 'Krakow': 0.769}
    df_output['population'] = df_output['city'].map(population)
    
    # rests_to_pop - отношение числа ресторанов в городе к численности населения
    df_output['rests_to_pop'] = df_output['rest_city'] / df_output['population']
    
    # ranked_cities - ранг городов
    df_output['ranked_cities'] = df_output['city'].rank()
 
    
    # dummy variable city_'name of city'
    df_output = pd.get_dummies(df_output, columns=['city',], dummy_na=True)
    
    
    
    # ################### 5. Clean #################################################### 
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object' or df_output[s].dtypes == 'datetime64[ns]' ]
    df_output.drop(object_columns, axis = 1, inplace=True)
    df_output = df_output.fillna(df_output.mean()) 
    
    return df_output



In [200]:
data = pd.read_csv('main_task_new.csv')

data = preproc_data(data)
data.sample(10)

,ranking,rating,price_range,number_of_reviews,id_ta,number_of_reviews_is_nan,cuisine_style_is_nan,price_range_is_nan,cuisine_style_count,review_days_range,...,city_Oporto,city_Oslo,city_Paris,city_Prague,city_Rome,city_Stockholm,city_Vienna,city_Warsaw,city_Zurich,city_nan
24425,12770.0,4.0,0.814046,6.0,10157986.0,0,1,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
7945,8659.0,3.5,1.000000,31.0,12823528.0,0,0,0,3,0.0,...,0,0,0,0,0,0,0,0,0,0
4147,1461.0,4.0,1.000000,30.0,4401392.0,0,0,0,3,0.0,...,0,0,0,0,0,0,0,0,0,0
22192,3065.0,5.0,0.000000,41.0,4827446.0,0,0,0,3,0.0,...,0,0,0,0,1,0,0,0,0,0
9186,48.0,4.5,2.000000,2541.0,2005950.0,0,0,0,6,0.0,...,0,0,0,0,0,0,0,0,0,0
9981,6739.0,4.0,0.814046,12.0,6697300.0,0,0,1,1,0.0,...,0,0,1,0,0,0,0,0,0,0
28633,634.0,5.0,2.000000,6.0,12696113.0,0,0,0,5,0.0,...,0,0,0,0,0,0,0,0,1,0
26898,871.0,4.5,1.000000,51.0,4453030.0,0,0,0,4,0.0,...,0,0,0,0,0,0,1,0,0,0
25571,7203.0,3.0,0.814046,27.0,7084576.0,0,0,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0
26,2765.0,5.0,0.814046,11.0,10060659.0,0,1,1,1,0.0,...,0,0,0,0,0,0,0,0,0,0


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [203]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['rating'], axis = 1)
y = data['rating']

In [204]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [205]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [206]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [207]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [208]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.20381249999999998
